Importing the required packages

In [2]:
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import random

In [3]:
YELP_TOKEN = "12zz7ZvufU8cPdMvZg4dKg4qg3AY8aBmrN6SUcs21q_x3KvSpj-SajPOLiOdm6_s6V2cOyQ9nq65T5p7BX1Ufrd2-8mmJo-Nx2n50HrOgfj7kJd5PCPX-ihHoPy8YXYx"

In [12]:
r = requests.get("https://api.yelp.com/v3/businesses/search?location=Toronto&limit=50&offset=0", headers={"Authorization": "Bearer %s" % YELP_TOKEN})

In [5]:
r.headers

{'Connection': 'keep-alive', 'content-type': 'application/json', 'ratelimit-dailylimit': '5000', 'x-routing-service': 'routing-main--useast1-5745c65d47-hjnw8; site=public_api_v3', 'ratelimit-remaining': '4946', 'ratelimit-resettime': '2021-12-18T00:00:00+00:00', 'x-b3-sampled': '0', 'x-zipkin-id': 'ae251be783dbef8e', 'server': 'envoy', 'x-cloudmap': 'routing_useast1', 'content-encoding': 'gzip', 'x-proxied': '10-65-85-173-useast1aprod', 'x-extlb': '10-65-85-173-useast1aprod', 'cache-control': 'max-age=0, no-store, private, no-transform', 'Accept-Ranges': 'bytes', 'Date': 'Fri, 17 Dec 2021 23:31:32 GMT', 'Via': '1.1 varnish', 'X-Served-By': 'cache-fty21350-FTY', 'X-Cache': 'MISS', 'X-Cache-Hits': '0', 'Vary': 'Accept-Encoding', 'transfer-encoding': 'chunked'}

In [6]:
print(r.status_code, r.reason, r.content)

200 OK b'{"businesses": [{"id": "r_BrIgzYcwo1NAuG9dLbpg", "alias": "pai-northern-thai-kitchen-toronto-5", "name": "Pai Northern Thai Kitchen", "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/cO0LcL_SGxZUYrqVeGPJOQ/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/pai-northern-thai-kitchen-toronto-5?adjust_creative=FtKJx3tKxygY3LUrNXmfyw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=FtKJx3tKxygY3LUrNXmfyw", "review_count": 2976, "categories": [{"alias": "thai", "title": "Thai"}], "rating": 4.5, "coordinates": {"latitude": 43.64784, "longitude": -79.38872}, "transactions": [], "price": "$$", "location": {"address1": "18 Duncan Street", "address2": "", "address3": "", "city": "Toronto", "zip_code": "M5H 3G8", "country": "CA", "state": "ON", "display_address": ["18 Duncan Street", "Toronto, ON M5H 3G8", "Canada"]}, "phone": "+14169014724", "display_phone": "+1 416-901-4724", "distance": 3009.3347434869856}, {"id": "iGEvDk6hsizigmXhDKs2Vg", "alias": 

In [ ]:
r.json()

Yelp has a limit of 50 reviews per request. 

In [ ]:
review_labels = []

limit = 50
offset_value = 5
offsets = [x for x in range(offset_value)]

for offset in offsets:
  api_url = "https://api.yelp.com/v3/businesses/search?location=Toronto&limit=50&offset={o}".format(o = offset)
  r = requests.get(api_url, headers={"Authorization": "Bearer %s" % YELP_TOKEN})

  for business in r.json()['businesses']:
    print(business['name'])
    reviews = requests.get("https://api.yelp.com/v3/businesses/%s/reviews" % business['id'], headers={"Authorization": "Bearer %s" % YELP_TOKEN}).json()
    review_url = reviews['reviews'][0]['url']
    page = requests.get(review_url)
    soup = bs(page.content, 'html.parser')
    allreview = soup.find_all("div", class_ = lambda value: value and value.startswith("review__"))
    for review in allreview:
      review_labels.append((review.find("span", class_ = lambda value: value and value.startswith("raw__")).text.rstrip('.'), reviews['reviews'][0]['rating']))

review_labels

We use limit and offset to go through different pages of businesses to increase the sample size of reviews. In addition, we use beautiful soup to scrape the urls of each business to extract the top 10 reviews as visible on the website

In [35]:
len(review_labels)

2400

In [ ]:
review_features = [(x.split(' '), 'positive' if y > 3 else 'negative') for (x, y) in review_labels]
review_features

In [22]:
from nltk.sentiment import SentimentAnalyzer
import nltk.sentiment.util
from nltk.classify import NaiveBayesClassifier

random.shuffle(review_features)
training_docs = review_features[:1500]
test_docs = review_features[1500:]

print("Training: %d, Testing: %d" % (len(training_docs), len(test_docs)))

sentim_analyzer = SentimentAnalyzer()

Training: 1500, Testing: 900


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
all_words_neg = sentim_analyzer.all_words([nltk.sentiment.util.mark_negation(doc) for doc in training_docs])
all_words_neg

In [24]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
sentim_analyzer.add_feat_extractor(nltk.sentiment.util.extract_unigram_feats, unigrams=unigram_feats)

In [25]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(test_docs)

In [26]:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
     print('{0}: {1}'.format(key, value))

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.9933333333333333
F-measure [negative]: 0.8235294117647058
F-measure [positive]: 0.9966024915062287
Precision [negative]: 1.0
Precision [positive]: 0.9932279909706546
Recall [negative]: 0.7
Recall [positive]: 1.0


We get a much higher accuracy with this dataset.